# CHAPTER 6 머신러닝을 이용한 투자 전략

## 6.1 ETFs를 활용한 주가 방향 예측
### 6.1.1 데이터
### 6.1.2 모델 학습
### 6.1.3 머신러닝 모델을 활용한 결과 예측
### 6.1.4 확장 가능한 방법

## 6.2 k-최근접 이웃 알고리즘을 활용한 투자 전략
### 6.2.1 데이터
### 6.2.2 모델 학습

## 6.3 클러스터링 알고리즘을 활용한 종목 분류
### 6.3.1 데이터
### 6.3.2 클러스터링

## 6.4 마치며